In [4]:
import pandas as pd
import json
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import FastText
from epochlogger import EpochLogger
from sentencegenerator import SentenceGenerator
import nltk
import re

ImportError: DLL load failed: The specified module could not be found.

In [53]:
data = pd.read_csv('../data/mfp-diaries-shuffled.tsv',sep='\t',header=None,chunksize=40000)

### Construct the corpus with each sentence corresponding to a diary entry

In [3]:
# i = 0
# with open("../data/sentences.txt", "a",encoding='utf-8') as sentences:
#     for chunk in data:
#         for row in chunk.iterrows():
#             ex_diary = json.loads(row[1][2])
#             for meal in ex_diary:
#                 dish_list = []
#                 for dish in meal['dishes']:
#                     if dish['name'][0:20] == 'Quick Added Calories':
#                         provider = 'Quick Added Calories'
#                         item = 'Quick Added Calories'
#                     else:
#                         name, quant = dish['name'].split(',',maxsplit=1)
#                         try:
#                             provider, item = name.split(' - ')[0:2]
#                         except ValueError:
#                             if len(name.split(' - ')) ==1:
#                                 provider = name
#                                 item = ""
#                     word = '_'.join(provider.split()) + '___' + '_'.join(item.split())
#                     dish_list.append(word)
#                 sentence = " ".join(dish_list)
#                 sentences.write(sentence+'\n')
#             i += 1
#             print(i,end='\r')

### Construct the corpus with each sentence corresponding to an individual food name

In [54]:
i = 0
with open("../data/foods_as_sentences_lower_nltk.txt", "a",encoding='utf-8') as sentences:
    for chunk in data:
        for row in chunk.iterrows():
            ex_diary = json.loads(row[1][2])
            for meal in ex_diary:
                for dish in meal['dishes']:
                    if dish['name'][0:20] == 'Quick Added Calories':
                        name = 'Quick Added Calories'
                    else:
                        name, quant = dish['name'].split(',',maxsplit=1)
                        tokens = re.split("[, \-!?*+()012345678~9&%=/\"#.>^<:]+",name)
                        tokens_lower =  [token.lower() for token in tokens if len(token) > 2]
                    sentence = ' '.join(tokens_lower)
                    sentences.write(sentence+'\n')
            i += 1
            print(i,end='\r')

In [42]:
sentences.close()

In [43]:
total_ex = i

## Train Word2Vec

In [9]:
epoch_logger = EpochLogger()
sentences = SentenceGenerator('../data/foods_as_sentences.txt')
model = Word2Vec(sentences,size=100,window=6,min_count=10)
model.train(sentences, total_examples=total_ex, epochs=10,callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end


(269294174, 295199880)

In [ ]:
model.save("../word2vecmodels/model_foods_as_sentences_FULL.model")

In [13]:
model.wv.index2word

['Cheese',
 'Chicken',
 'Raw',
 'Generic',
 'Milk',
 '&',
 'Coffee',
 'Chocolate',
 'Butter',
 'and',
 'With',
 'Bread',
 'Homemade',
 'Protein',
 'Egg',
 'Organic',
 'Cream',
 'Whole',
 'Light',
 'White',
 'Fresh',
 'Free',
 'Sugar',
 'Peanut',
 'Salad',
 'Yogurt',
 'Bar',
 'Breast',
 'Almond',
 'Sweet',
 'Fat',
 'Rice',
 'Eggs',
 'Natural',
 'Turkey',
 'Apple',
 'Vanilla',
 'Greek',
 'Green',
 'Oil',
 'Banana',
 'Value',
 'Red',
 'Calories',
 'Quick',
 'Great',
 'Tesco',
 'Original',
 'Added',
 'Wheat',
 'Fruit',
 'Kroger',
 'Honey',
 'Trader',
 'Brewed',
 '100%',
 'Sauce',
 'Beef',
 'Large',
 'Tea',
 'Kraft',
 'Bacon',
 'Cheddar',
 'Potato',
 'from',
 'grounds',
 'Spinach',
 'Chips',
 'Roasted',
 'Creamer',
 'Medium',
 'Brown',
 'Black',
 'Farms',
 'Dressing',
 'Tomato',
 'Half',
 'Beans',
 'Oats',
 'Frozen',
 'Soup',
 'Baby',
 'Coconut',
 'the',
 'Whey',
 'Mix',
 'Broccoli',
 'French',
 'Olive',
 'Pizza',
 'Baked',
 'Kirkland',
 '1',
 'Sausage',
 'Oatmeal',
 'Carrots',
 'Dark',
 'P

In [11]:
model.wv.most_similar("6")

[('(Raspberries', 0.9843159914016724),
 ('Rasberries', 0.9639889597892761),
 ('Raspberriess', 0.9560129642486572),
 ('raspberries', 0.951819658279419),
 ('Fruit/Raspberries', 0.9398398995399475),
 ('Bluberries', 0.9139211177825928),
 ('Straberries', 0.9107884168624878),
 ('Mulberries', 0.9078725576400757),
 ('Strwberries', 0.8971955180168152),
 ('bluberries', 0.8908132314682007)]

In [15]:
model.wv.most_similar('Coffee')

[('Coffe', 0.6892114877700806),
 ('Cappuccino', 0.6448010206222534),
 ('Tea', 0.6218572854995728),
 ('coffee', 0.5886411070823669),
 ('Decaf', 0.5647045373916626),
 ('Americano', 0.563005805015564),
 ('Latte', 0.5549725294113159),
 ('Coffee-', 0.5541614294052124),
 ('Folgers', 0.5238036513328552),
 ('Espresso', 0.5066592693328857)]

In [16]:
model.wv.most_similar('Beer')

[('Lager', 0.682033360004425),
 ('Nastro', 0.6280746459960938),
 ('Ale', 0.6231441497802734),
 ('Brewery', 0.5951564311981201),
 ('Shiner', 0.5871891975402832),
 ('Beer:', 0.5701658725738525),
 ('Rootbeer', 0.5633116960525513),
 ('Azzurro', 0.5631240606307983),
 ('Azzuro', 0.5569137334823608),
 ('Ipa', 0.5559419989585876)]

### Train FastText

In [55]:
sentences = SentenceGenerator('../data/foods_as_sentences_lower_nltk.txt')
model = FastText(sentences,min_count=10,size=200)

In [56]:
model.save("../fasttextmodels/model_full_fasttext_lower_nltk_200.model")

In [57]:
model.wv.index2word

['cheese',
 'chicken',
 'raw',
 'milk',
 'chocolate',
 'generic',
 'coffee',
 'butter',
 'with',
 'and',
 'egg',
 'whole',
 'bread',
 'protein',
 'cream',
 'homemade',
 'free',
 'fat',
 'organic',
 'white',
 'light',
 'sugar',
 'vanilla',
 'peanut',
 'fresh',
 'yogurt',
 'bar',
 'salad',
 'breast',
 'almond',
 'sweet',
 'natural',
 'rice',
 'original',
 'turkey',
 'eggs',
 'apple',
 'greek',
 'green',
 'oil',
 'banana',
 'red',
 'value',
 'large',
 'great',
 'wheat',
 'fruit',
 'tesco',
 'medium',
 'honey',
 'from',
 'sauce',
 'kroger',
 'beef',
 'cheddar',
 'tea',
 'trader',
 'bacon',
 'potato',
 'brewed',
 'half',
 'black',
 'roasted',
 'kraft',
 'spinach',
 'frozen',
 'dressing',
 'chips',
 'creamer',
 'brown',
 'grounds',
 'small',
 'tomato',
 'oats',
 'farms',
 'beans',
 'soup',
 'aldi',
 'the',
 'baby',
 'mix',
 'coconut',
 'strawberry',
 'baked',
 'whey',
 'french',
 'dark',
 'broccoli',
 'unsweetened',
 'cooked',
 'sausage',
 'pizza',
 'olive',
 'oatmeal',
 'low',
 'grain',
 'g

In [58]:
model.wv.similarity('Coffee','coffee')

0.82400707544173346

In [67]:
model.wv.most_similar(positive=['corona','light','ale'])

[('corale', 0.5726001858711243),
 ('maiale', 0.5720107555389404),
 ('budlight', 0.5637447834014893),
 ('male', 0.538725733757019),
 ('tale', 0.5315029621124268),
 ("ti'light", 0.517282247543335),
 ('female', 0.5147026777267456),
 ("light'n", 0.5124111771583557),
 ('[light', 0.5067524313926697),
 ('aleppo', 0.5016406178474426)]

In [17]:
oov_word = "fishhh" 
oov_word in model.wv.vocab

False

In [18]:
vector = model.wv['cake'] 

In [ ]:
from gensim.models import FastText
model = FastText.load('../')

In [22]:
model.wv.similar_by_vector(oov_vec)

'fish'